# Manipulating Excel Workbooks with Python

In this demo, we will show that Python (via [`openpyxl`](https://openpyxl.readthedocs.io/en/stable/))can be used to both read and write to Excel workbooks. `openpyxl` allows not only data interaction by cell and range, but also embedding of formulas and the construction of charts. To demonstrate these features, we will do the following:

1. Construct a very simple Income Statement class
2. Create a sheet that holds income data for the first year
3. Project the income into the future based upon a couple growth parameters for revenue and cost of goods sold. *The projection will rely on embedded Excel formulas and not Python-only functionality.*
4. Produce a chart that shows a stream of earnings flows through the projection window.

In [3]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
import openpyxl as xl
from dataclasses import dataclass
import seaborn as sb

%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Define an Income Statement 

Our bare bones income statement will have no nuance whatsoever, and encompasses only the path from revenues to net earnings. We will assume the following:

1. The statement covers one year, and the fiscal year coincides with the calendar year. (Consequently, we will only represent the period by year.)
2. Revenues in year 0 are 100. Prices are constant every year at \$1 per unit, only unit volume changes.
3. The cost of producing each unit is constant at \$0.80.
4. The firm has an outstanding bond of 100 payable at 2% in every year we care about.
5. The firm has a tax rate of 10%, applied on earnings after interest.

This parsimony is not required, the same kind of operations could occur with a much more realistic model of income.

In [7]:
@dataclass
class IncomeStatement:
    period: int
    revenues: float
    cogs: float
    interest: float
    tax_rate: float
        
    def __post_init__(self) -> None:
        self.ebit: float = self.revenues + self.cogs
        self.pretax: float = self.ebit + self.interest
        self.tax: float = (self.tax_rate * self.pretax) * -1
        self.net_earnings: float = self.pretax + self.tax
            
            
start: IncomeStatement = IncomeStatement(
    period=0,
    revenues=100,
    cogs=-80,
    interest=2,
    tax_rate=0.1
)
    
start.__dict__

{'period': 0,
 'revenues': 100,
 'cogs': -80,
 'interest': 2,
 'tax_rate': 0.1,
 'ebit': 20,
 'pretax': 22,
 'tax': -2.2,
 'net_earnings': 19.8}

## Generate Workbook

Workbooks can either be created or loaded. Here we are going to create one programmatically:

In [8]:
!rm income_statement.xlsx

rm: income_statement.xlsx: No such file or directory


In [10]:
class IncomeWorkbook:
        
    def __init__(self, file_name: str = "income_statement.xlsx") -> None:
        self.file_name: str = file_name
        self.wb: xl.Workbook = xl.Workbook()
        self.model = self.wb.active
        self.unit_data = self.wb.create_sheet("unit_data", 0)
        self.model.title = "model"
        self.unit_data.title = "unit_data"
        
    def gen_unit_data(self, num_years: int = 20, init_val: float = 100.) -> None:
        self.unit_data["A1"] = "Years"
        self.unit_data["A2"] = "Units"
        self.unit_data["A3"] = "=A1+A2"
#         self.unit_data["A4"] = self.unit_data.ce

    def save(self) -> None:
        self.wb.save(self.file_name)
    
test: IncomeWorkbook = IncomeWorkbook()

In [13]:
test.gen_unit_data()
test.save()